# Chapter 14. Inheritance: For Better or for Worse

- The super() function

- The pitfalls (cạm bẫy) of subclassing from built-in types

- Multiple inheritance and method resolution order

- Mix classes

(Multiple inheritance - chỉ ở C++ - được cho là trouble hơn là có ích)
(Đồng thời nói về việc lạm dụng kế thừa --> 1 thay đổi nhỏ khiến hệ thống dễ vỡ + khó hiểu)

## The super() Function

Khi một lớp con ghi đè một phương thức của một lớp cha, phương thức ghi đè thường cần gọi phương thức từ lớp cha

```python
class LastUpdatedOrderedDict(OrderedDict):
    """Store items in the order they were last updated"""

    def __setitem__(self, key, value):
        super().__setitem__(key, value)
        # OrderedDict.__setitem__(self, key, value) ko đc recommend - khó maintain + ko giúp multiple inheritance
        self.move_to_end(key)
```



Gọi `__init__` bị ghi đè là đặc biệt quan trọng để cho phép lớp cha thực hiện phần của chúng trong việc khởi tạo.

```python
    def __init__(self, a, b) :
        super().__init__(a, b) # (khác vs Java chỉ cần super() )
        ...  # more initialization code
```

## Subclassing Built-In Types Is Tricky

Lý do:
The code of the built-ins (written in C) usually does not call methods overridden by user-defined classes

In [1]:
# Example 14-1. Our __setitem__ override is ignored by the __init__ and __update__ methods of the built-in dict

class DoppelDict(dict):
    def __setitem__(self, key, value):
        super().__setitem__(key, [value] * 2)

dd = DoppelDict(one=1) # init
dd

{'one': 1}

In [3]:
dd['two'] = 2 # set, okay, it worked
dd

{'one': 1, 'two': [2, 2]}

In [4]:
dd.update(three=3) # update
dd

{'one': 1, 'two': [2, 2], 'three': 3}

--> This is what is called “late binding"

In [5]:
# Example 14-2. The __getitem__ of AnswerDict is bypassed by dict.update

class AnswerDict(dict):
    def __getitem__(self, key):
        return 42

ad = AnswerDict(a='foo')
ad['a'] # ok

42

In [6]:
d = {}
d.update(ad) # The dict.update method ignored AnswerDict.__getitem__
d

{'a': 'foo'}

--> Thay vì subclass từ build-in (`dict`, `list`, `str`), sử dụng từ collections module (`UserDict`, `UserList`, and `UserString`)

Giờ thì, tập trung vào multiple inheritance, các mà Python quyết định dùng attr nào cho `super().attr`

## Multiple Inheritance and Method Resolution Order

Bất kỳ ngôn ngữ nào hỗ trợ Đa kế thừa đều phải giải quyết "diamon problem"

![](image/14-1.png)

Figure 14-1.
- Left: Activation sequence for the `leaf1.ping()` call.
- Right: Activation sequence for the `leaf1.pong()` call.


In [7]:
# Example 14-4. diamond.py: classes Leaf, A, B, Root form the graph in Figure 14-1

# tag::DIAMOND_CLASSES[]
class Root:  # <1>
    def ping(self):
        print(f'{self}.ping() in Root')

    def pong(self):
        print(f'{self}.pong() in Root')

    def __repr__(self):
        cls_name = type(self).__name__
        return f'<instance of {cls_name}>'


class A(Root):  # <2>
    def ping(self):
        print(f'{self}.ping() in A')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in A')
        super().pong()


class B(Root):  # <3>
    def ping(self):
        print(f'{self}.ping() in B')
        super().ping()

    def pong(self):
        print(f'{self}.pong() in B')


class Leaf(A, B):  # <4>
    def ping(self):
        print(f'{self}.ping() in Leaf')
        super().ping()
# end::DIAMOND_CLASSES[]

In [8]:
leaf1 = Leaf()
leaf1.ping()
# Vì ping ở Leaf, A, B đề gọi super()

<instance of Leaf>.ping() in Leaf
<instance of Leaf>.ping() in A
<instance of Leaf>.ping() in B
<instance of Leaf>.ping() in Root


In [9]:
leaf1.pong()
# Leaf ko thấy pong(), gọi pong qua A, rồi gọi tiếp B

<instance of Leaf>.pong() in A
<instance of Leaf>.pong() in B


Activation sequences để gọi đó được quyết định bởi:
- The method resolution order of the `Leaf` class.
- The use of `super()` in each method.

Class nào cũng nó `__mro__` lưu tuple các ref tới superclass, xác định thứ tự kích hoạt

In [10]:
Leaf.__mro__

(__main__.Leaf, __main__.A, __main__.B, __main__.Root, object)

Python is a dynamic language,
--> so the interaction of `super()` with the `MRO` is also dynamic

In [11]:
# Example 14-6. diamond2.py: classes to demonstrate the dynamic nature of super()
# tag::DIAMOND_CLASSES[]

# from diamond import A  # <1>

class U():  # <2>
    def ping(self):
        print(f'{self}.ping() in U')
        super().ping()  # <3>

class LeafUA(U, A):  # <4>
    def ping(self):
        print(f'{self}.ping() in LeafUA')
        super().ping()
# end::DIAMOND_CLASSES[]

class LeafAU(A, U):
    def ping(self):
        print(f'{self}.ping() in LeafAU')
        super().ping()

In [12]:
u = U()
u.ping()

<__main__.U object at 0x7fd6da19ec50>.ping() in U


AttributeError: 'super' object has no attribute 'ping'

In [13]:
U.__mro__ # object ko có ping

(__main__.U, object)

In [14]:
leaf2 = LeafUA()
leaf2.ping()

<instance of LeafUA>.ping() in LeafUA
<instance of LeafUA>.ping() in U
<instance of LeafUA>.ping() in A
<instance of LeafUA>.ping() in Root


Trong một chương trình thực tế, một lớp như U có thể là một lớp mixin: một lớp dự định được sử dụng cùng với các lớp khác trong đa kế thừa, để cung cấp chức năng bổ sung.

In [16]:
# Ví dụ thứ tự kích hoạt ở thư viện tkinter

import tkinter

def print_mro(cls):
    print(', '.join(c.__name__ for c in cls.__mro__))

print_mro(tkinter.Text)

Text, Widget, BaseWidget, Misc, Pack, Place, Grid, XView, YView, object


![](image/14-2.png)

## Mixin Classes

- Được thiết kế để subclass cùng với ít nhất một lớp khác trong đa kế thừa.
- Không được coi là base class vì nó ko cung cấp đủ chức năng

### Case-Insensitive Mappings


In [17]:
# Example 14-8: một lớp được thiết kế để cung cấp quyền truy cập không phân biệt chữ hoa chữ thường
# tag::UPPERCASE_MIXIN[]
import collections

def _upper(key):  # <1>
    try:
        return key.upper()
    except AttributeError:
        return key

class UpperCaseMixin:  # <2> Mixin class
    def __setitem__(self, key, item):
        super().__setitem__(_upper(key), item)

    def __getitem__(self, key):
        return super().__getitem__(_upper(key))

    def get(self, key, default=None):
        return super().get(_upper(key), default)

    def __contains__(self, key):
        return super().__contains__(_upper(key))
# end::UPPERCASE_MIXIN[]

# tag::UPPERDICT[]
class UpperDict(UpperCaseMixin, collections.UserDict):  # <1>
    pass

class UpperCounter(UpperCaseMixin, collections.Counter):  # <2>
    """Specialized 'Counter' that uppercases string keys"""  # <3>
# end::UPPERDICT[]


In [18]:
d = UpperDict([('a', 'letter A'), (2, 'digit two')])
list(d.keys())

['A', 2]

In [20]:
d['b'] = 'letter B'
'b' in d

True

In [21]:
d['a'], d.get('B')

('letter A', 'letter B')

In [22]:
d

{'A': 'letter A', 2: 'digit two', 'B': 'letter B'}

In [23]:
c = UpperCounter('BaNanA')
c

UpperCounter({'B': 1, 'A': 3, 'N': 2})

In [24]:
c.most_common()

[('A', 3), ('N', 2), ('B', 1)]

## Multiple Inheritance in the Real World

Trong C++, ví dụ duy nhất của đa kế thừa nằm ở Adapter pattern


### ABCs Are Mixins Too

Ở Python, đa kế thừa nằm ở việc dùng `collections.abc`.
- Xét cho cùng, Java cũng có đa kế thùa interfaces
- Và, ABCs là các khai báo interfaces, để imple các method cụ thể

"Mixin method"
- Method impl nhiều collection ABCs

VD: Việc impl `collections.UserDict` dựa vào một số mixin methods được cung cấp bởi `collections.abc.MutableMapping`

### ThreadingMixIn and ForkingMixIn

The http.server package provides `HTTPServer` and `ThreadingHTTPServer` classes.

In [ ]:
from http import server

# t = server.ThreadingHTTPServer()

"""
class ThreadingHTTPServer(socketserver.ThreadingMixIn, HTTPServer):
    daemon_threads = True

Lớp này giống với HTTPServer nhưng sử dụng các thread để xử lý các yêu cầu bằng cách sử dụng ThreadingMixIn.
Điều này rất hữu ích để xử lý pre-opening sockets của web browsers, mà HTTPServer sẽ đợi vô thời hạn.
"""

In [27]:
class ThreadingMixIn:
    """Mixin class to handle each request in a new thread."""

    # 8 lines omitted in book listing

    def process_request_thread(self, request, client_address):  # 1
        ... # 6 lines omitted in book listing

    def process_request(self, request, client_address):  #2
        ... # 8 lines omitted in book listing

    def server_close(self):  # 3
        super().server_close()
        self._threads.join()

# 1. Ko call super() vì nó là method mới, ko phải override
# 2. Overrides `process_request` mà HTTPServer kế thừa từ `socketserver.BaseServer`,
# bắt đầu 1 thread và uỷ thác cho process_request_thread đang chạy trong luồng đó.
# Nó không gọi super().

# 3. calls super().server_close() để dừng nhận requests. Sau đó chờ threads (cái mà process_request start) để dừng

### Django Generic Views Mixins

In Django, a __view__ is a callable object that takes a `request` argument—an object representing an HTTP request—and returns an object representing an HTTP response (có thể có nhiều loại response)

Originally, Django provided a set of functions, called __generic views__, that implemented some common use cases

- The original generic views were functions, so they were not extensible.

 Figure 14-3. At the top of the diagram we see two classes that take care of very distinct responsibilities: `View` and `TemplateResponseMixin`.

![](image/14-3.png)


- `View` is the base class of all views (it could be an `ABC`), and it provides core functionality like the `dispatch` method, to “handler” methods like `get`, `head`, `post`, etc., implemented by concrete subclasses to handle the different HTTP verbs.
    - Vì `RedirectView` chỉ thực hiện impl handler methods, tại sao các methods đó ko đặt trong View interface ?
        - Subclasses are free to implement just the handlers they want to support
                - VD: A `TemplateView` is used only to display content, so it only implements get

- `TemplateResponseMixin` cung cấp chức năng cho views cần dùng template.
    - `RedirectView` ko có content body -> no need of a template and it does not inherit from this mixin
    - `TemplateResponseMixin` provides behaviors to `TemplateView` and other template-rendering views, such as `ListView`, `DetailView`, etc.,

 Figure 14-4 depicts the django.views.generic.list module and part of the base module.

![](image/14-4.png)


### Multiple Inheritance in Tkinter

Figure 14-5. Summary UML diagram for the Tkinter GUI class hierarchy;

classes tagged «mixin» are designed to provide concrete methods to other classes via multiple inheritance.

![](image/14-5.png)


## Coping with Inheritance

"Vẫn chưa có lý thuyết chung nào về tính kế thừa có thể hướng dẫn các lập trình viên thực hành."

Sau đây là 1 số mẹo nhỏ để tránh "spaghetti class graphs."

### Ưu tiên Thành phần đối tượng hơn Kế thừa lớp

- Đó là second principle của OOP design trong quyển Design Patterns

"Favoring composition" leads to more flexible designs
- VD: `tkinter.Widget` class,
    - thay về kế thừa method từ all geometry managers
    - widget instances could hold a reference to a geometry manager, and invoke its methods.
    - could add a new geometry manager without touching the widget class hierarchy and without worrying about name clashes

### Understand Why Inheritance Is Used in Each Case

The main reasons are:

- Inheritance of interface creates a subtype, implying an “is-a” relationship. This is best done with ABCs.

- Inheritance of implementation avoids code duplication by reuse. Mixins can help with this.

### Make Interfaces Explicit with ABCs

In modern Python, if a class is intended to define an interface, it should be an explicit ABC or a typing.Protocol subclass. An ABC should subclass only abc.ABC or other ABCs. Multiple inheritance of ABCs is not problematic.

### Use Explicit Mixins for Code Reuse

If a class is designed to provide method implementations for reuse by multiple unrelated subclasses, without implying an “is-a” relationship, it should be an explicit mixin class

### Provide Aggregate Classes to Users

Một lớp được xây dựng chủ yếu bằng cách kế thừa từ mixin và không thêm cấu trúc hoặc hành vi riêng của nó được gọi là aggregate class.

```python
class ListView(MultipleObjectTemplateResponseMixin, BaseListView):
    """
    Render some list of objects, set by `self.model` or `self.queryset`.
    `self.queryset` can actually be any iterable of items, not just a queryset.
    """
```

### Subclass Only Classes Designed for Subclassing

WARNING

Subclassing any complex class and overriding its methods is error-prone because the superclass methods may ignore the subclass overrides in unexpected ways. As much as possible, avoid overriding methods, or at least restrain yourself to subclassing classes which are designed to be easily extended, and only in the ways in which they were designed to be extended.

### Avoid Subclassing from Concrete Classes

Subclassing concrete classes is more dangerous than subclassing ABCs and mixins, because instances of concrete classes usually have internal state that can easily be error when you override methods that depend on that state

https://realpython.com/inheritance-composition-python/